In [19]:
#import libraries

import datetime
import io
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import pickle
import random
import requests
import yfinance as yf

plt.style.use('fivethirtyeight')

In [20]:
from get_data import get_index_constituents
from get_data import get_stock_data

In [21]:
sp500 = get_index_constituents('sp500')
ftse250 = get_index_constituents('ftse250')
ftse100 = get_index_constituents('ftse100')
stoxx = get_index_constituents('stoxx')
cac_40 = get_index_constituents('cac_40')
russell = get_index_constituents('russell')
dax = get_index_constituents('dax')

aex = get_index_constituents('aex')
asx = get_index_constituents('asx')

ftsemib = get_index_constituents('ftsemib')
ibexnr = get_index_constituents('ibexnr')

crypto = get_index_constituents('crypto')
fx = get_index_constituents('fx')

In [22]:
str_convert_dict = {"aex": aex,
                    "asx": asx,
                    "cac_40": cac_40,
                    "crypto": crypto,
                    "dax": dax,
                    "stoxx": stoxx,
                    "ftse100": ftse100,
                    "ftse250": ftse250,
                    "ftsemib": ftsemib,
                    "fx": fx,
                    "ibexnr": ibexnr,
                    "sp500": sp500,
                    "russell": russell
                   }

In [23]:
def add_daily_data(stock, index):
    # to get ticker
    ticker = yf.Ticker(stock)
        
    if stock == 'CON.DE':
        return
    
    try:
        data = pd.read_csv(r".\yfinance\{}\1D\{}.csv".format(index, stock))
    except FileNotFoundError:
        data = ticker.history(period='max', interval="1D")
        data.index.names = ['']
        if data.empty:
            print("No daily data for {} in index {} on yfinance. Delete/update the ticker.".format(stock, index))
            return
        data.to_csv(r"C:\systematic\shru_strat\yfinance/{}/1D/{}.csv".format(index, stock), mode='w', header=True)
        print("New ticker for daily data: {} in index {}".format(stock, index))
        return
    # assuming that CON.DE can either be not present or be present and not empty
    if data.empty:
        data = ticker.history(period='max', interval="1D")
        if data.empty:
            print(f"No daily data for '{stock}' in '{index}' on yfinance. Delete/update the ticker.")
            return 
        data.to_csv(r".\yfinance/{}/1D/{}.csv".format(index, stock), mode='a+', header=True)
        print(f"Updating daily data for '{stock}' in '{index}' which earlier was empty")
        return
    
    last_timestamp = data.iloc[-50, :]['Unnamed: 0']

    data_to_add = ticker.history(period='max', interval="1D").loc[last_timestamp:,:]
    data = data.set_index('Unnamed: 0').loc[:last_timestamp,:]
        
    pd.concat([data, data_to_add]).to_csv(f"./{index}/1D/{stock}.csv", mode='w', header=True)
    print(f"Updating daily data for index: '{index}', stock: {stock}")    
    
    return

In [24]:
# for index in str_convert_dict():
#     for stock in str_convert_dict[index]:
#         print(stock)
#         add_daily_data(stock, index)

In [25]:
def get_hourly_data(stock):
    # to get ticker
    ticker = yf.Ticker(stock)
    
    # get data for last 730 days
    start = (datetime.date.today()-datetime.timedelta(729)).strftime("%Y-%m-%d")
    end = datetime.date.today().strftime("%Y-%m-%d")

    data = ticker.history(start=start, end=end, interval="1H")
    
    return data

In [26]:
def add_hourly_data(stock, index): 
    # to get ticker
    ticker = yf.Ticker(stock)
        
    if stock == 'CON.DE':
        return
    
    try:
        data = pd.read_csv(r".\yfinance\{}\1H\{}.csv".format(index, stock))
    except FileNotFoundError:
        data = get_hourly_data(stock).rename({'Datetime': 'Unnamed: 0'}, axis=1)
        if data.empty:
            print("No hourly data for {} in index {} on yfinance. Delete/update the ticker.".format(stock, index))
            return
        data.to_csv(r"C:\systematic\shru_strat\yfinance/{}/1H/{}.csv".format(index, stock), mode='w', header=True)
        print("New ticker for hourly data: {} in index {}".format(stock, index))
        return
    
    # assuming that CON.DE can either be not present or be present and not empty
    if data.empty:
        data = get_hourly_data(stock)
        if data.empty:
            print(f"No hourly data for '{stock}' in '{index}' on yfinance. Delete/update the ticker.")
            return 
        data.to_csv(r".\yfinance\{}\1H\{}.csv".format(index, stock), mode='w', header=True)
        print(f"Updating hourly data for '{index}': {stock}' which earlier was empty")
        return
    
    last_timestamp = data.iloc[-10, :]['Unnamed: 0']

    start = last_timestamp.split()[0]
    start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = (datetime.date.today()).strftime("%Y-%m-%d")
    
    days_to_fetch_data = datetime.datetime.today() - start_date
    if days_to_fetch_data > datetime.timedelta(729):
        last_missed_date = (datetime.date.today()-datetime.timedelta(730)).strftime("%Y-%m-%d")
        data_to_add = get_hourly_data(stock)
        print(f"You wouldn't get continuous data. Data not available from {start} to {last_missed_date}")
    else:
        data_to_add = ticker.history(start=start, interval="1H").loc[last_timestamp:,:]
        data = data.set_index('Unnamed: 0').loc[:last_timestamp,:]
    
    pd.concat([data, data_to_add]).to_csv(f"./{index}/1H/{stock}.csv", mode='w', header=True)
    print(f"Updating hourly data for index: '{index}', stock: {stock}")
    
    return

In [27]:
# for index in str_convert_dict.keys():
#     for stock in str_convert_dict[index]:
#         print(stock)
#         add_hourly_data(stock, index)

In [28]:
def get_5_minutewise_data(stock):
    # to get ticker
    ticker = yf.Ticker(stock)
    
    # get data for last 60 days
    start = (datetime.date.today()-datetime.timedelta(59)).strftime("%Y-%m-%d")
    end = datetime.date.today().strftime("%Y-%m-%d")

    data = ticker.history(start=start, end=end, interval="5m")
    data.index.names = ['']
    
    return data

In [29]:
def add_5_minutewise_data(stock, index):
    # to get ticker
    ticker = yf.Ticker(stock)
        
    if stock == 'CON.DE':
        return
    
    try:
        data = pd.read_csv(r".\yfinance\{}\5m\{}.csv".format(index, stock))
    except FileNotFoundError:
        data = get_5_minutewise_data(stock)
        if data.empty:
            print("No 5 minutewise data for {} in index {} on yfinance. Delete/update the ticker.".format(stock, index))
            return
        data.to_csv(r".\yfinance/{}/5m/{}.csv".format(index, stock), mode='w', header=True)
        print("New ticker for 5 minutewise data: {} in index {}".format(stock, index))
        return
    
    # assuming that CON.DE can either be not present or be present and not empty
    if data.empty:
        data = get_5_minutewise_data(stock)
        if data.empty:
            print(f"No 5 minutewise data for '{stock}' in '{index}' on yfinance. Delete/update the ticker.")
            return 
        data.to_csv(r"C:\systematic\shru_strat\yfinance\{}\5m\{}.csv".format(index, stock), mode='w', header=True)
        print(f"Updating 5 minutewise data for '{index}': {stock}' which earlier was empty")
        return
    
    last_timestamp = data.iloc[-10, :]['Unnamed: 0']

    start = last_timestamp.split()[0]
    start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = (datetime.date.today()).strftime("%Y-%m-%d")
    
    days_to_fetch_data = datetime.datetime.today() - start_date
    if days_to_fetch_data > datetime.timedelta(59):
        last_missed_date = (datetime.date.today()-datetime.timedelta(60)).strftime("%Y-%m-%d")
        data_to_add = get_5_minutewise_data(stock)
        print(f"You wouldn't get continuous data. Data not available from {start} to {last_missed_date}")
    else:
        data_to_add = ticker.history(start=start, interval="5m").loc[last_timestamp:,:]
        data_to_add.index.names = ['']
        data = data.set_index('Unnamed: 0').loc[:last_timestamp,:]
            
    pd.concat([data, data_to_add]).to_csv(f"./{index}/5m/{stock}.csv", mode='w', header=True)
    print(f"Updating 5 minutewise data for index: '{index}', stock: {stock}")
    
    return

In [30]:
# for index in str_convert_dict.keys():
#     for stock in str_convert_dict[index]:
#         print(stock)
#         add_5_minutewise_data(stock, index)

In [31]:
def get_minutewise_data(stock, i=0):
    # to get ticker
    ticker = yf.Ticker(stock)
    
    # get data for last 5 days and do this for 6 times to get
    start = (datetime.date.today()-datetime.timedelta(i+4)).strftime("%Y-%m-%d")
    end = (datetime.date.today()-datetime.timedelta(i)).strftime("%Y-%m-%d")

    data = ticker.history(start=start, end=end, interval="1m")
    data.index.names = ['']
    return data

In [32]:
def add_minutewise_data(stock, index):              
    # to get ticker
    ticker = yf.Ticker(stock)
    
    if stock == 'CON.DE':
        return
    
    try:
        data = pd.read_csv(r".\yfinance\{}\1m\{}.csv".format(index, stock))
    except FileNotFoundError:
        data = get_minutewise_data(stock)
        if data.empty:
            print("No minutewise data for {} in index {} on yfinance. Delete/update the ticker.".format(stock, index))
            return
        data.to_csv(r".\shru_strat\yfinance/{}/1m/{}.csv".format(index, stock), mode='w', header=True)
        print("New ticker for minutewise data: {} in index {}".format(stock, index))
        return
    
    # assuming that CON.DE can either be not present or be present and not empty
    if data.empty:
        data = get_minutewise_data(stock)
        if data.empty:
            print(f"No minutewise data for '{stock}' in '{index}' on yfinance. Delete/update the ticker.")
            return 
        data.to_csv(r".\shru_strat\yfinance\{}\1m\{}.csv".format(index, stock), mode='w', header=True)
        print(f"Updating minutewise data for '{index}': {stock}' which earlier was empty")
        return

    
    last_timestamp = data.iloc[-10, :]['Unnamed: 0']

    start = last_timestamp.split()[0]
    start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = (datetime.date.today()).strftime("%Y-%m-%d")
    
    data_to_add = pd.DataFrame()
    
    days_to_fetch_data = datetime.datetime.today() - start_date
    if days_to_fetch_data > datetime.timedelta(29):
        last_missed_date = str(datetime.date.today()-datetime.timedelta(30))
        data_to_add = get_minutewise_data(stock)
        print(f"You wouldn't get continuous data. Data not available from {start} to {last_missed_date}")
    else:
        for i in range(0, 29, 7):
            new_start_date = start_date + datetime.timedelta(days=i)
            new_end_date = start_date + datetime.timedelta(days=i+6)
            if new_end_date <= datetime.datetime.today():
                tmp_data = ticker.history(start=new_start_date.strftime("%Y-%m-%d"), \
                                          end=new_end_date.strftime("%Y-%m-%d"), interval="1m")
                data_to_add = data_to_add.append(tmp_data)
            else:
                tmp_data = ticker.history(start=new_start_date.strftime("%Y-%m-%d"), end=end, interval="1m")
                data_to_add = data_to_add.append(tmp_data.loc[last_timestamp:, :].iloc[1:,:])
                data_to_add.index.names = ['']
                break
            
        data = data.set_index('Unnamed: 0').loc[:last_timestamp,:]    

    pd.concat([data, data_to_add]).to_csv(f"./{index}/1m/{stock}.csv", mode='w', header=True)
    print(f"Updating minutewise data for index: '{index}', stock: {stock}")
    
    return

In [34]:
add_minutewise_data('AAP', 'sp500')

You wouldn't get continuous data. Data not available from 2022-07-21 to 2023-02-11
Updating minutewise data for index: 'sp500', stock: AAP


In [16]:
# for index in str_convert_dict.keys():
#     for stock in str_convert_dict[index]:
#         print(index, stock)
#         add_minutewise_data(stock, index)

In [17]:
# indices = ['^AEX', '^AXJO', '^FCHI', '^GDAXI', '^DJI', '^STOXX', 'FTAD.L', '^FTSE', '^FTMC', 'FTSEMIB.MI', '^IBEX', '^NSEI', '^GSPC', '^VIX', 'IEMG', 'EEM', 'KBE']
# for ticker in indices:
#     # ticker = '^RUT'
#     index = 'indices'

#     add_daily_data(ticker, index)
#     add_hourly_data(ticker, index)
#     add_5_minutewise_data(ticker, index)
#     add_minutewise_data(ticker, index)

In [18]:
currencies = ['GBP', 'EUR', 'CHF', 'RMB', 'AUD', 'USD', 'CAD', '']
for currency in currencies:
    if currency in ('USD', ''):
        continue
    for currency2 in currencies:
        if currency == currency2:
            continue
        index = 'fx'
        ticker = currency + currency2 + '=X'
        
        print(ticker)
        
        add_daily_data(ticker, index)
        print('1D')
        add_hourly_data(ticker, index)
        print('1H')
        add_5_minutewise_data(ticker, index)
        print('5m')
        add_minutewise_data(ticker, index)
        print('1m')

GBPEUR=X
Updating daily data for stock: GBPEUR=X in index: 'fx'
1D
Updating hourly data for index: 'fx', stock: GBPEUR=X
1H
New ticker for 5 minutewise data: GBPEUR=X in index fx
5m
New ticker for minutewise data: GBPEUR=X in index fx
1m
GBPCHF=X
Updating daily data for stock: GBPCHF=X in index: 'fx'
1D
Updating hourly data for index: 'fx', stock: GBPCHF=X
1H
New ticker for 5 minutewise data: GBPCHF=X in index fx
5m
New ticker for minutewise data: GBPCHF=X in index fx
1m
GBPRMB=X
- GBPRMB=X: No data found, symbol may be delisted
No daily data for GBPRMB=X in index fx on yfinance. Delete/update the ticker.
1D
- GBPRMB=X: No data found, symbol may be delisted
No hourly data for GBPRMB=X in index fx on yfinance. Delete/update the ticker.
1H
- GBPRMB=X: No data found, symbol may be delisted
No 5 minutewise data for GBPRMB=X in index fx on yfinance. Delete/update the ticker.
5m
- GBPRMB=X: No data found, symbol may be delisted
No minutewise data for GBPRMB=X in index fx on yfinance. Delete/

- RMB=X: No data found, symbol may be delisted
No minutewise data for RMB=X in index fx on yfinance. Delete/update the ticker.
1m
AUDGBP=X
New ticker for daily data: AUDGBP=X in index fx
1D
New ticker for hourly data: AUDGBP=X in index fx
1H
New ticker for 5 minutewise data: AUDGBP=X in index fx
5m
New ticker for minutewise data: AUDGBP=X in index fx
1m
AUDEUR=X
New ticker for daily data: AUDEUR=X in index fx
1D
New ticker for hourly data: AUDEUR=X in index fx
1H
New ticker for 5 minutewise data: AUDEUR=X in index fx
5m
New ticker for minutewise data: AUDEUR=X in index fx
1m
AUDCHF=X
New ticker for daily data: AUDCHF=X in index fx
1D
New ticker for hourly data: AUDCHF=X in index fx
1H
New ticker for 5 minutewise data: AUDCHF=X in index fx
5m
New ticker for minutewise data: AUDCHF=X in index fx
1m
AUDRMB=X
- AUDRMB=X: No data found, symbol may be delisted
No daily data for AUDRMB=X in index fx on yfinance. Delete/update the ticker.
1D
- AUDRMB=X: No data found, symbol may be delisted
No

In [21]:
others = ['BABA', 'SI', 'TSM']
index = 'others'
for ticker in others:
    add_daily_data(ticker, index)
    add_hourly_data(ticker, index)
    add_5_minutewise_data(ticker, index)
    add_minutewise_data(ticker, index)

New ticker for daily data: BABA in index others
Updating daily data for stock: SI in index: 'others'
New ticker for daily data: TSM in index others


In [49]:
add_daily_data('BTC-USD', 'crypto')

Updating daily data for index: 'crypto', stock: BTC-USD


In [53]:
others = str_convert_dict['crypto']
index = 'crypto'
for ticker in others:
    add_daily_data(ticker, index)
#     add_hourly_data(ticker, index)
#     add_5_minutewise_data(ticker, index)
#     add_minutewise_data(ticker, index)

New ticker for daily data: BTC-AUD in index crypto
New ticker for daily data: BTC-CAD in index crypto
New ticker for daily data: BTC-EUR in index crypto
New ticker for daily data: BTC-GBP in index crypto
New ticker for daily data: BTC-USD in index crypto
New ticker for daily data: DOGE-AUD in index crypto
New ticker for daily data: DOGE-CAD in index crypto
New ticker for daily data: DOGE-EUR in index crypto
New ticker for daily data: DOGE-GBP in index crypto
New ticker for daily data: DOGE-USD in index crypto
New ticker for daily data: ETH-AUD in index crypto
New ticker for daily data: ETH-CAD in index crypto
New ticker for daily data: ETH-EUR in index crypto
New ticker for daily data: ETH-GBP in index crypto
New ticker for daily data: ETH-USD in index crypto
